<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/ICP_best_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [214]:
import numpy as np
from scipy.spatial import KDTree

DX = 10
DY = 10
ANGLE = np.radians(-180)
NOISE = 0.05
N = 40
ITERS = 50
ANGLE_STEP = 10

v = np.array([[0,0],[0,4],[10,4],[10,0]])
points = np.vstack([v[i]+t*(v[(i+1)%4]-v[i]) for i in range(4) for t in np.linspace(0,1,N)])[:-1]
R_true = np.array([[np.cos(ANGLE), -np.sin(ANGLE)], [np.sin(ANGLE), np.cos(ANGLE)]])
points2 = points @ R_true + [DX, DY] + np.random.randn(*points.shape) * NOISE

def icp(src, tgt):
    tree = KDTree(tgt)
    R, t = np.eye(2), np.zeros(2)
    for _ in range(ITERS):
        tr = src @ R.T + t
        idx = tree.query(tr)[1]
        src_m, tgt_m = tr.mean(0), tgt[idx].mean(0)
        H = (tr - src_m).T @ (tgt[idx] - tgt_m)
        U, _, Vt = np.linalg.svd(H)
        R_new = Vt.T @ U.T
        t_new = tgt_m - R_new @ src_m
        R, t = R_new @ R, R_new @ t + t_new
    return t[0], t[1], np.arctan2(R[1,0], R[0,0])

best_dx, best_dy, best_angle = 0, 0, 0
best_score = float('inf')

for deg in np.arange(-180, 181, ANGLE_STEP):
    rad = np.radians(deg)
    R_init = np.array([[np.cos(rad), -np.sin(rad)], [np.sin(rad), np.cos(rad)]])

    dx, dy, da = icp(points @ R_init.T, points2)

    R_total = np.array([[np.cos(da), -np.sin(da)], [np.sin(da), np.cos(da)]]) @ R_init
    total_angle_rad = np.arctan2(R_total[1,0], R_total[0,0])
    total_angle_deg = np.degrees(total_angle_rad)

    transformed = (points @ R_init.T) @ np.array([[np.cos(da), -np.sin(da)], [np.sin(da), np.cos(da)]]) + [dx, dy]
    score = np.median(np.linalg.norm(points2 - transformed, axis=1))
    if score < best_score:
        best_score = score
        best_dx, best_dy, best_angle = dx, dy, total_angle_deg

print(f"Истинные параметры:")
print(f"  Угол: {np.degrees(ANGLE):.1f}°")
print(f"  Смещение: ({DX:.1f}, {DY:.1f})")
print()
print(f"Найденные параметры (лучший результат):")
print(f"  Угол: {best_angle:.4f}°")
print(f"  Смещение: ({best_dx:.4f}, {best_dy:.4f})")
print(f"  Качество (медианная ошибка): {best_score:.6f}")

Истинные параметры:
  Угол: -180.0°
  Смещение: (10.0, 10.0)

Найденные параметры (лучший результат):
  Угол: -179.8721°
  Смещение: (9.9956, 10.0125)
  Качество (медианная ошибка): 2.022671
